In [ ]:
from ortools.sat.python import cp_model

# Create model
model = cp_model.CpModel()

# Map professors to integers
PROFESSORS = {'A': 0, 'B': 1, 'C': 2}
prof_names = ['A', 'B', 'C']

# Classes and time slots (start, end)
classes = {
    1: (8, 9),
    2: (8.5, 9.5),
    3: (9, 10),
    4: (9, 10),
    5: (10.5, 11.5),
}

# Class to allowed professors mapping
allowed_profs = {
    1: [PROFESSORS['A'], PROFESSORS['C']],
    2: [PROFESSORS['A']],
    3: [PROFESSORS['B'], PROFESSORS['C']],
    4: [PROFESSORS['B'], PROFESSORS['C']],
    5: [PROFESSORS['A'], PROFESSORS['B']],
}

# Step 1: Define Variables
class_vars = {}
for c in classes:
    class_vars[c] = model.NewIntVarFromDomain(
        cp_model.Domain.FromValues(allowed_profs[c]), f'class_{c}'
    )

print(class_vars)

{1: class_1(0, 2), 2: class_2(0), 3: class_3(1..2), 4: class_4(1..2), 5: class_5(0..1)}


In [5]:
# Step 2: Add Constraints for overlapping classes
def times_overlap(time1, time2):
    return not (time1[1] <= time2[0] or time2[1] <= time1[0])

# For each pair of overlapping classes, if same prof assigned, constraint violation
for c1 in classes:
    for c2 in classes:
        if c1 < c2 and times_overlap(classes[c1], classes[c2]):
            model.Add(class_vars[c1] != class_vars[c2])

# Step 3: Solve the problem
solver = cp_model.CpSolver()

class CSPPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, prof_names):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.variables = variables
        self.prof_names = prof_names
        self.solution_count = 0

    def on_solution_callback(self):
        self.solution_count += 1
        print(f'\nSolution {self.solution_count}:')
        for c in sorted(self.variables):
            assigned = self.Value(self.variables[c])
            print(f'Class {c} -> Professor {self.prof_names[assigned]}')

# Print all solutions
# solution_printer = CSPPrinter(class_vars, prof_names)
# status = solver.SearchForAllSolutions(model, solution_printer)

# Optionally limit to first solution by using:
solver.Solve(model)
for c in sorted(class_vars):
    print(f'Class {c} → Professor {prof_names[solver.Value(class_vars[c])]}')


Class 1 → Professor C
Class 2 → Professor A
Class 3 → Professor C
Class 4 → Professor B
Class 5 → Professor A
